In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import os
import torch
import tqdm
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision
from IPython import display
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid, save_image
from timm.models.vision_transformer import VisionTransformer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Load Data

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    )
])

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
valid_dataset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=transform)
display.clear_output()

In [ ]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Visualization

In [ ]:
data_iter = iter(train_loader)
images, labels = next(data_iter)

img_grid = make_grid(images, nrow=8)

plt.figure(figsize=(8, 8))
plt.imshow(img_grid.permute(1, 2, 0))
plt.axis('off')
plt.show()

# ViT

In [ ]:
model = VisionTransformer(
    img_size=224,
    patch_size=16,
    in_chans=3,
    embed_dim=768,
    depth=12,
    num_heads=12,
    mlp_ratio=4,
    qkv_bias=True,
    norm_layer=nn.LayerNorm,
    num_classes=10
).to('cuda')

# Train

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
n_epochs = 10

In [ ]:
train_losses = []
valid_losses = []

In [ ]:
for epoch in tqdm.tqdm(range(n_epochs)):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, label in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
    epoch_train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)
    epoch_train_accuracy = correct / total
    
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
    epoch_valid_loss = running_loss / len(valid_loader.dataset)
    valid_losses.append(epoch_valid_loss)
    epoch_valid_accuracy = correct / total
    
    print(f"Epoch [{epoch}/{n_epochs}]: Train Loss: {epoch_train_loss:.3f}, Train Accuracy: {epoch_train_accuracy:.3f}%")
    print(f"Epoch [{epoch}/{n_epochs}]: Valid Loss: {epoch_valid_loss:.3f}, Valid Accuracy: {epoch_valid_accuracy:.3f}%")
    print()

# Results

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(train_losses)
plt.plot(valid_losses)
plt.title("Loss Curve")
plt.legend(["Train Loss", "Valid Loss"])
plt.show()